In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from my_setting import my_pd_setting, my_np_setting, my_sns_setting, data_path
from my_stat import missing
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

my_np_setting()
my_sns_setting()
my_pd_setting()

numpy的版本为1.20.3
seaborn的版本为: 0.11.2
pandas的版本为: 1.3.4


In [2]:
filepath = r'C:\Users\86199\Documents\Datas\Mechine_learing\book1'
data_path(filepath)

当前数据路径为: C:\Users\86199\Documents\Datas\Mechine_learing\book1


['boston_house_prices.csv', 'wdbc.csv', 'wdbc.txt', 'wine.csv', 'wine.txt']

In [3]:
df = pd.read_csv('wdbc.csv', header=None)
df.head(2)
X = df.loc[:, 2:].values
y = df.loc[:, 1].values
la = LabelEncoder()
y_l = la.fit_transform(y)
print(f'''y包含的类标签为:  {la.classes_}''')
print(f'''列表['M', 'B']的编码情况为:   {la.transform(['M','B'])}''')

y包含的类标签为:  ['B' 'M']
列表['M', 'B']的编码情况为:   [1 0]


In [4]:

X_train,X_test,y_train,y_test = train_test_split(X,y_l,test_size=0.2,random_state=1, stratify=y)
pipe_lr = make_pipeline(
    StandardScaler(),
    PCA(n_components=2),
    LogisticRegression(random_state=1, solver='lbfgs')
)
pipe_lr.fit(X_train,y_train)
y_pred = pipe_lr.predict(X_test)
print(f'精度为: {pipe_lr.score(X_test, y_test)}')


精度为: 0.956140350877193


In [5]:
scores = cross_val_score(estimator=pipe_lr,
                         X=X_train,
                         y=y_train,
                         cv=10,
                         n_jobs=1)
print('CV accuracy scores: %s' % scores)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

CV accuracy scores: [0.93478261 0.93478261 0.95652174 0.95652174 0.93478261 0.95555556
 0.97777778 0.93333333 0.95555556 0.95555556]
CV accuracy: 0.950 +/- 0.014


In [6]:
import scipy
scipy.__version__

'1.7.1'

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

pipe_svc = make_pipeline(StandardScaler(),
                         SVC(random_state=1))

param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

param_grid = [{'svc__C': param_range, 
               'svc__kernel': ['linear']},
              {'svc__C': param_range, 
               'svc__gamma': param_range, 
               'svc__kernel': ['rbf']}]

gs = GridSearchCV(estimator=pipe_svc, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  refit=True,
                  cv=10,
                  n_jobs=-1)
gs = gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

0.9846859903381642
{'svc__C': 100.0, 'svc__gamma': 0.001, 'svc__kernel': 'rbf'}
